# Библиотеки и функции

In [ ]:
import pandas as pd
from graphlib.structures import Graph
from graphlib.algorithms import DFS_with_cc, BFS_search
from graphlib.tools import radius_approximate, diameter_approximate

# Загрузка данных

# Выделение компонент связности, характеристики наибольшей компоненты

# Метрические характеристики графа: диаметр, радиус, 90 процентиль (approximation)